In [65]:
import pandas as pd
import numpy as np
import math
import pickle

Read the book sent over by Westley and Stephanie

In [55]:
pd.options.display.max_columns = 50
dat_orig = pd.read_excel("2018_sales.xlsx", header = 1)
dat_orig.fillna(method='ffill', inplace=True)
dat_orig = dat_orig.filter(regex=r'(Wk.|Unnamed: 0|Unnamed: 1|Unnamed: 2)')
dat_orig.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Feb Wk. 1 2019,Feb Wk. 2 2019,Feb Wk. 3 2019,Feb Wk. 4 2019,Mar Wk. 5 2019,Mar Wk. 6 2019,Mar Wk. 7 2019,Mar Wk. 8 2019,Mar Wk. 9 2019,Apr Wk. 10 2019,Apr Wk. 11 2019,Apr Wk. 12 2019,Apr Wk. 13 2019,May Wk. 14 2019,May Wk. 15 2019,May Wk. 16 2019,May Wk. 17 2019,Jun Wk. 18 2019,Jun Wk. 19 2019,Jun Wk. 20 2019,Jun Wk. 21 2019,Jun Wk. 22 2019,...,Aug Wk. 28 2019,Aug Wk. 29 2019,Aug Wk. 30 2019,Sep Wk. 31 2019,Sep Wk. 32 2019,Sep Wk. 33 2019,Sep Wk. 34 2019,Sep Wk. 35 2019,Oct Wk. 36 2019,Oct Wk. 37 2019,Oct Wk. 38 2019,Oct Wk. 39 2019,Nov Wk. 40 2019,Nov Wk. 41 2019,Nov Wk. 42 2019,Nov Wk. 43 2019,Dec Wk. 44 2019,Dec Wk. 45 2019,Dec Wk. 46 2019,Dec Wk. 47 2019,Dec Wk. 48 2019,Jan Wk. 49 2019,Jan Wk. 50 2019,Jan Wk. 51 2019,Jan Wk. 52 2019
0,Direct,Sls Total FP Rtl,LY,2.6191e+06,3.11482e+06,1.26302e+06,3.10634e+06,4.94786e+06,4.48451e+06,1.99622e+06,6.10595e+06,4.65447e+06,3.4179e+06,2.12765e+06,6.02738e+06,4.56318e+06,6.46613e+06,3.41393e+06,2.93784e+06,4.20344e+06,2.78572e+06,4.24735e+06,2.99027e+06,1.95681e+06,1.80254e+06,...,2.19364e+06,1.81738e+06,3.37993e+06,3.81247e+06,4.72602e+06,5.48783e+06,2.97255e+06,5.34449e+06,4.57149e+06,3.99308e+06,4.0983e+06,5.41473e+06,5.39372e+06,4.27014e+06,7.18727e+06,1.04607e+07,5.25634e+06,4.21524e+06,2.35427e+06,1.51287e+06,1.65111e+06,3.40117e+06,1.73363e+06,773961,2.12887e+06
1,Direct,Sls Total MD Rtl,LY,418697,1.28096e+06,2.86295e+06,1.49025e+06,1.09141e+06,1.85633e+06,2.34381e+06,1.21474e+06,1.76859e+06,613653,1.55391e+06,1.23227e+06,1.86073e+06,1.30668e+06,1.14679e+06,2.1711e+06,1.99288e+06,1.30856e+06,809572,727637,3.25918e+06,3.70596e+06,...,1.60537e+06,1.29413e+06,2.24379e+06,1.83228e+06,630941,424986,635694,820040,775590,1.00772e+06,1.01295e+06,902646,939332,1.66965e+06,1.00221e+06,1.57471e+06,964273,743994,965570,2.14527e+06,2.19711e+06,1.87944e+06,1.42259e+06,2.46773e+06,591970
2,Direct,Sls Total Rtl,LY,3.03779e+06,4.39577e+06,4.12597e+06,4.59658e+06,6.03927e+06,6.34084e+06,4.34003e+06,7.32069e+06,6.42306e+06,4.03155e+06,3.68156e+06,7.25965e+06,6.42391e+06,7.77282e+06,4.56072e+06,5.10894e+06,6.19632e+06,4.09429e+06,5.05692e+06,3.7179e+06,5.21599e+06,5.5085e+06,...,3.79901e+06,3.11151e+06,5.62372e+06,5.64475e+06,5.35696e+06,5.91282e+06,3.60824e+06,6.16453e+06,5.34708e+06,5.0008e+06,5.11125e+06,6.31738e+06,6.33306e+06,5.93978e+06,8.18947e+06,1.20354e+07,6.22062e+06,4.95924e+06,3.31984e+06,3.65814e+06,3.84822e+06,5.28061e+06,3.15622e+06,3.24169e+06,2.72084e+06
3,Direct,Sls Total FP U,LY,45660,59187,18413,50013,96095,88177,35225,114460,88038,64415,37244,103743,76577,131041,69266,46402,69532,48981,90057,65149,35758,28060,...,38540,38177,59254,58620,81046,101669,50305,90661,78539,64949,62570,79031,88187,70103,138995,232955,111309,92066,48745,21558,24758,66300,31738,11443,32693
4,Direct,Sls Total MD U,LY,6235,42613,116496,51186,31134,72162,97720,45198,79963,19158,46561,37299,72394,43733,37296,78712,78946,41226,24728,20221,125874,149246,...,73411,60906,92483,90384,13947,13202,14457,23533,25386,31463,24829,27926,33080,55953,34526,63617,35631,27840,34742,78473,92472,81460,68134,115223,19307


In [40]:
h1 = [1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6]
h2 = [x+6 for x in h1]

months = h1 + h2
weeks = list(range(1, 53))

month_map = pd.DataFrame()
month_map['FISCAL_MO'] = months
month_map['FISCAL_WK'] = weeks



In [41]:
def format_sales(df):
    
    ids = list(df.iloc[:, np.r_[0:2]].columns)
    vals = list(df.iloc[:,3:].columns)
    
    melt = pd.melt(df, id_vars = ids, value_vars = vals, var_name = 'FISCAL_WK', value_name = 'SALES')
    
    melt.rename(columns={'Unnamed: 0':'Channel',
                      'Unnamed: 1': 'summation_type'
                          
                          }, 
                 inplace=True)
    
    sub = melt[melt['summation_type'] == "Sls Total Rtl"]
    sub[['Channel', 'SALES']].groupby('Channel').agg('sum')
    
    sub.FISCAL_WK = sub.FISCAL_WK.str[8:]
    sub.FISCAL_WK = sub.FISCAL_WK.str[:2]
    sub.FISCAL_WK = sub.FISCAL_WK.str.strip()
    
    h1 = [1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6]
    h2 = [x+6 for x in h1]

    months = h1 + h2
    weeks = list(range(1, 53))
    qtrs = newList = [math.ceil(x / 3) for x in months]

    month_map = pd.DataFrame()
    month_map['FISCAL_MO'] = months
    month_map['FISCAL_WK'] = weeks
    month_map['FISCAL_QTR'] = qtrs
    
    sub.FISCAL_WK = sub.FISCAL_WK.astype(str).astype(int)
    sales_fcst = pd.merge(sub, month_map, on='FISCAL_WK', how='inner')
    
    return sales_fcst

In [42]:
fcst = format_sales(dat_orig)
fcst.head()

/Users/jj5jxt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,Channel,summation_type,FISCAL_WK,SALES,FISCAL_MO,FISCAL_QTR
0,Direct,Sls Total Rtl,1,3.03779e+06,1,1
1,Retail,Sls Total Rtl,1,5.16914e+06,1,1
2,Direct,Sls Total Rtl,2,4.39577e+06,1,1
3,Retail,Sls Total Rtl,2,5.44481e+06,1,1
4,Direct,Sls Total Rtl,3,4.12597e+06,1,1


In [43]:
fcst.SALES.sum()

672500672.3399997

In [44]:
# 2018 customer data to estimate pct of sales by customer type and $_p_cust
qtr_data = pd.read_csv('quarterly_sales_to_custs.csv').sort_values('Channel')
qtr_data.head()

,FISCAL_YR,Channel,total_net_sales,total_custs,FISCAL_QTR,recency_net_sales_total,recency_cust_total,Recency,chan_rec_sales_pct,chan_rec_cust_pct,chan_rec_seg_sales,chan_rec__seg_custs,Customer_Type,Segment,chan_rec_sales,chan_rec__custs,sales_pct,cust_pct,d_p_cust
0,2018,D,5.721810e+07,401995,1,3.675641e+06,36585,0,0.064239,0.091009,3.297407e+06,31814,NTB_1,1,3.877800e+06,34250,0.850329,0.928876,103.646405
39,2018,D,5.933927e+07,447606,4,6.467598e+06,67558,2,0.108994,0.150932,1.077873e+06,5712,React__Omni,_Omni,6.856067e+06,63402,0.157214,0.090092,188.703201
38,2018,D,5.933927e+07,447606,4,6.467598e+06,67558,2,0.108994,0.150932,5.778195e+06,57690,React_1,1,6.856067e+06,63402,0.842786,0.909908,100.159382
37,2018,D,5.602950e+07,377032,3,6.236244e+06,55832,2,0.111303,0.148083,1.010387e+06,4236,React__Omni,_Omni,6.553415e+06,52434,0.154177,0.080787,238.523885
36,2018,D,5.602950e+07,377032,3,6.236244e+06,55832,2,0.111303,0.148083,5.543028e+06,48198,React_1,1,6.553415e+06,52434,0.845823,0.919213,115.005353


In [45]:
direct_total = fcst[['Channel', 'SALES', 'FISCAL_QTR']].query('Channel == "Direct"').groupby(['FISCAL_QTR', 'Channel']).agg('sum').reset_index()
retail_total = fcst[['Channel', 'SALES', 'FISCAL_QTR']].query('Channel == "Retail"').groupby(['FISCAL_QTR', 'Channel']).agg('sum').reset_index()
qtr_sales = pd.concat([direct_total, retail_total])
qtr_sales['Channel'] = np.where(qtr_sales['Channel']=='Direct', 'D', 'R')

In [46]:
qtr_sales.SALES.sum()

672500672.3400002

In [47]:
qtr_sales = qtr_sales.rename(index=str, columns={"SALES": "channel_qtr_sales"})

In [48]:
qtr_dat = pd.merge(qtr_data, qtr_sales, on = ['FISCAL_QTR', 'Channel'], how = 'inner')
qtr_dat.head()

,FISCAL_YR,Channel,total_net_sales,total_custs,FISCAL_QTR,recency_net_sales_total,recency_cust_total,Recency,chan_rec_sales_pct,chan_rec_cust_pct,chan_rec_seg_sales,chan_rec__seg_custs,Customer_Type,Segment,chan_rec_sales,chan_rec__custs,sales_pct,cust_pct,d_p_cust,channel_qtr_sales
0,2018,D,5.721810e+07,401995,1,3.675641e+06,36585,0,0.064239,0.091009,3.297407e+06,31814,NTB_1,1,3.877800e+06,34250,0.850329,0.928876,103.646405,68016682.15
1,2018,D,5.721810e+07,401995,1,6.414162e+06,60238,2,0.112100,0.149848,1.074583e+06,4675,React__Omni,_Omni,6.781020e+06,56169,0.158469,0.083231,229.857276,68016682.15
2,2018,D,5.721810e+07,401995,1,6.414162e+06,60238,2,0.112100,0.149848,5.706438e+06,51494,React_1,1,6.781020e+06,56169,0.841531,0.916769,110.817524,68016682.15
3,2018,D,5.721810e+07,401995,1,4.712819e+07,305170,1,0.823659,0.759139,1.807705e+07,52128,Existing__Omni,_Omni,5.305491e+07,262268,0.340723,0.198759,346.781892,68016682.15
4,2018,D,5.721810e+07,401995,1,4.712819e+07,305170,1,0.823659,0.759139,3.497786e+07,210140,Existing_1,1,5.305491e+07,262268,0.659277,0.801242,166.450267,68016682.15


In [56]:
qtr_dat['rec_seg_exp_sales'] = qtr_dat.sales_pct * qtr_dat.channel_qtr_sales * qtr_dat.chan_rec_sales_pct
qtr_dat.head()

,FISCAL_YR,Channel,total_net_sales,total_custs,FISCAL_QTR,recency_net_sales_total,recency_cust_total,Recency,chan_rec_sales_pct,chan_rec_cust_pct,chan_rec_seg_sales,chan_rec__seg_custs,Customer_Type,Segment,chan_rec_sales,chan_rec__custs,sales_pct,cust_pct,d_p_cust,channel_qtr_sales,rec_seg_exp_sales,implied_custs
0,2018,D,5.721810e+07,401995,1,3.675641e+06,36585,0,0.064239,0.091009,3.297407e+06,31814,NTB_1,1,3.877800e+06,34250,0.850329,0.928876,103.646405,68016682.15,3.715371e+06,35846.598152
1,2018,D,5.721810e+07,401995,1,6.414162e+06,60238,2,0.112100,0.149848,1.074583e+06,4675,React__Omni,_Omni,6.781020e+06,56169,0.158469,0.083231,229.857276,68016682.15,1.208278e+06,5256.643947
2,2018,D,5.721810e+07,401995,1,6.414162e+06,60238,2,0.112100,0.149848,5.706438e+06,51494,React_1,1,6.781020e+06,56169,0.841531,0.916769,110.817524,68016682.15,6.416408e+06,57900.661316
3,2018,D,5.721810e+07,401995,1,4.712819e+07,305170,1,0.823659,0.759139,1.807705e+07,52128,Existing__Omni,_Omni,5.305491e+07,262268,0.340723,0.198759,346.781892,68016682.15,1.908819e+07,55043.795479
4,2018,D,5.721810e+07,401995,1,4.712819e+07,305170,1,0.823659,0.759139,3.497786e+07,210140,Existing_1,1,5.305491e+07,262268,0.659277,0.801242,166.450267,68016682.15,3.693435e+07,221894.219494


In [58]:
qtr_dat['back_d_p_cust'] = qtr_dat.rec_seg_exp_sales / qtr_dat.chan_rec__seg_custs

In [61]:
qtr_dat.chan_rec__seg_custs.sum()

3435910

In [62]:
qtr_dat['implied_custs_back'] = qtr_dat.rec_seg_exp_sales / qtr_dat.back_d_p_cust

In [63]:
qtr_dat[['FISCAL_QTR', 'implied_custs_back']].groupby('FISCAL_QTR').agg('sum') # 2018 quarterly customers

,implied_custs_back
FISCAL_QTR,
1,831070.0
2,895229.0
3,806497.0
4,903114.0


In [67]:
to_join = qtr_dat[['d_p_cust', 'back_d_p_cust']]
df1 = pd.read_pickle('quarter_2019')
df2 = pd.merge(df1, to_join, on = 'd_p_cust', how = 'inner')
df2['back_implied_custs'] = df2.rec_seg_exp_sales / df2.back_d_p_cust


In [68]:
df2[['FISCAL_QTR', 'back_implied_custs']].groupby('FISCAL_QTR').agg('sum') # 2019 quartlerly customers

,back_implied_custs
FISCAL_QTR,
1,825282.467654
2,894067.594025
3,829088.144171
4,933825.948487


In [69]:
df2.head()

,FISCAL_YR,Channel,total_net_sales,total_custs,FISCAL_QTR,recency_net_sales_total,recency_cust_total,Recency,chan_rec_sales_pct,chan_rec_cust_pct,chan_rec_seg_sales,chan_rec__seg_custs,Customer_Type,Segment,chan_rec_sales,chan_rec__custs,sales_pct,cust_pct,d_p_cust,channel_qtr_sales,rec_seg_exp_sales,implied_custs,back_d_p_cust,back_implied_custs
0,2018,D,5.721810e+07,401995,1,3.675641e+06,36585,0,0.064239,0.091009,3.297407e+06,31814,NTB_1,1,3.877800e+06,34250,0.850329,0.928876,103.646405,70927548.79,3.874375e+06,37380.702190,116.784153,33175.523502
1,2018,D,5.721810e+07,401995,1,6.414162e+06,60238,2,0.112100,0.149848,1.074583e+06,4675,React__Omni,_Omni,6.781020e+06,56169,0.158469,0.083231,229.857276,70927548.79,1.259988e+06,5481.609191,258.455156,4875.072998
2,2018,D,5.721810e+07,401995,1,6.414162e+06,60238,2,0.112100,0.149848,5.706438e+06,51494,React_1,1,6.781020e+06,56169,0.841531,0.916769,110.817524,70927548.79,6.691007e+06,60378.599053,124.604962,53697.755932
3,2018,D,5.721810e+07,401995,1,4.712819e+07,305170,1,0.823659,0.759139,1.807705e+07,52128,Existing__Omni,_Omni,5.305491e+07,262268,0.340723,0.198759,346.781892,70927548.79,1.990510e+07,57399.469748,366.179242,54358.888826
4,2018,D,5.721810e+07,401995,1,4.712819e+07,305170,1,0.823659,0.759139,3.497786e+07,210140,Existing_1,1,5.305491e+07,262268,0.659277,0.801242,166.450267,70927548.79,3.851501e+07,231390.485127,175.760693,219133.227785


In [72]:
df2[['rec_seg_exp_sales','FISCAL_QTR']].groupby('FISCAL_QTR').agg('sum') # 2019 quarterly net sales (fcst'd)

,rec_seg_exp_sales
FISCAL_QTR,
1,1.733915e+08
2,1.674980e+08
3,1.728449e+08
4,1.663969e+08


In [73]:
qtr_dat[['rec_seg_exp_sales','FISCAL_QTR']].groupby('FISCAL_QTR').agg('sum') #2018 actualized quarterly net sales

,rec_seg_exp_sales
FISCAL_QTR,
1,1.751573e+08
2,1.678743e+08
3,1.684817e+08
4,1.609631e+08


In [74]:
df2.rec_seg_exp_sales.sum() # 2019 forecasted net sales

680131282.4783108

In [75]:
qtr_dat.rec_seg_exp_sales.sum() # 2018 actual net sales

672476459.750431

In [50]:
qtr_dat['implied_custs'] = qtr_dat.rec_seg_exp_sales / qtr_dat.d_p_cust
qtr_dat.implied_custs.sum()

3443858.5823689965

In [51]:
qtr_dat[['FISCAL_QTR', 'implied_custs']].groupby('FISCAL_QTR').agg('sum')

,implied_custs
FISCAL_QTR,
1,848720.188165
2,875485.285220
3,810828.027239
4,908825.081744


In [52]:
qtr_dat.FISCAL_QTR.unique()

array([1, 4, 3, 2])

In [64]:
qtr_data['channel_year_sales'] = np.where(qtr_data['Channel']=='Direct', direct_total, retail_total)
yearly_data.head()

,FISCAL_YR,Channel,total_net_sales,total_custs,recency_net_sales_total,recency_cust_total,Recency,chan_rec_sales_pct,chan_rec_cust_pct,chan_rec_seg_sales,chan_rec__seg_custs,Customer_Type,Segment,chan_rec_sales,chan_rec__custs,sales_pct,cust_pct,d_p_cust,channel_year_sales
0,2018,D,2.298271e+08,1000034,2.127757e+07,166170,0,0.092581,0.166164,1.596978e+07,128114,NTB_1,1,2.214648e+07,145758,0.721098,0.878950,124.652897,2.652020e+08
1,2018,D,2.298271e+08,1000034,2.127757e+07,166170,0,0.092581,0.166164,6.176697e+06,17644,NTB__Omni,_Omni,2.214648e+07,145758,0.278902,0.121050,350.073527,2.652020e+08
4,2018,D,2.298271e+08,1000034,1.719479e+08,565710,1,0.748162,0.565691,8.828625e+07,250434,Existing_1,1,1.956016e+08,402343,0.451357,0.622439,352.532998,2.652020e+08
5,2018,D,2.298271e+08,1000034,1.719479e+08,565710,1,0.748162,0.565691,1.073154e+08,151909,Existing__Omni,_Omni,1.956016e+08,402343,0.548643,0.377561,706.445284,2.652020e+08
8,2018,D,2.298271e+08,1000034,3.660097e+07,268150,2,0.159254,0.268141,2.716340e+07,193009,React_1,1,3.868480e+07,228667,0.702173,0.844061,140.736445,2.652020e+08


In [67]:
yearly_data['rec_seg_exp_sales'] = yearly_data.sales_pct * yearly_data.channel_year_sales * yearly_data.chan_rec_sales_pct
yearly_data.head()

,FISCAL_YR,Channel,total_net_sales,total_custs,recency_net_sales_total,recency_cust_total,Recency,chan_rec_sales_pct,chan_rec_cust_pct,chan_rec_seg_sales,chan_rec__seg_custs,Customer_Type,Segment,chan_rec_sales,chan_rec__custs,sales_pct,cust_pct,d_p_cust,channel_year_sales,rec_seg_exp_sales
0,2018,D,2.298271e+08,1000034,2.127757e+07,166170,0,0.092581,0.166164,1.596978e+07,128114,NTB_1,1,2.214648e+07,145758,0.721098,0.878950,124.652897,2.652020e+08,1.770483e+07
1,2018,D,2.298271e+08,1000034,2.127757e+07,166170,0,0.092581,0.166164,6.176697e+06,17644,NTB__Omni,_Omni,2.214648e+07,145758,0.278902,0.121050,350.073527,2.652020e+08,6.847770e+06
4,2018,D,2.298271e+08,1000034,1.719479e+08,565710,1,0.748162,0.565691,8.828625e+07,250434,Existing_1,1,1.956016e+08,402343,0.451357,0.622439,352.532998,2.652020e+08,8.955564e+07
5,2018,D,2.298271e+08,1000034,1.719479e+08,565710,1,0.748162,0.565691,1.073154e+08,151909,Existing__Omni,_Omni,1.956016e+08,402343,0.548643,0.377561,706.445284,2.652020e+08,1.088584e+08
8,2018,D,2.298271e+08,1000034,3.660097e+07,268150,2,0.159254,0.268141,2.716340e+07,193009,React_1,1,3.868480e+07,228667,0.702173,0.844061,140.736445,2.652020e+08,2.965595e+07


In [69]:
yearly_data.chan_rec__seg_custs.sum()

1919447

In [70]:
yearly_data.rec_seg_exp_sales.sum()

672478325.0160993

In [72]:
yearly_data['back_d_p_cust'] = yearly_data.rec_seg_exp_sales / yearly_data.chan_rec__seg_custs
yearly_data[['d_p_cust', 'back_d_p_cust']]

,d_p_cust,back_d_p_cust
0,124.652897,138.195917
1,350.073527,388.107549
4,352.532998,357.601761
5,706.445284,716.602622
8,140.736445,153.650629
9,323.108383,352.757213
2,167.490932,156.847412
3,513.220373,480.606758
6,418.646433,415.517125
7,834.819306,828.579129


In [73]:
# using 2018 rec_seg_exp_sales
yearly_data['implied_custs'] = yearly_data.rec_seg_exp_sales / yearly_data.back_d_p_cust
yearly_data.implied_custs.sum()

1919447.0

In [75]:
year_hist = yearly_data.copy()

In [78]:
year_hist.implied_custs.sum()

1919447.0

In [87]:
yearly_data.head()
to_join = yearly_data[['d_p_cust', 'back_d_p_cust']]

In [79]:
df1 = pd.read_pickle('quarter_2019')

In [80]:
df1.head()

,FISCAL_YR,Channel,total_net_sales,total_custs,FISCAL_QTR,recency_net_sales_total,recency_cust_total,Recency,chan_rec_sales_pct,chan_rec_cust_pct,chan_rec_seg_sales,chan_rec__seg_custs,Customer_Type,Segment,chan_rec_sales,chan_rec__custs,sales_pct,cust_pct,d_p_cust,channel_qtr_sales,rec_seg_exp_sales,implied_custs
0,2018,D,5.721810e+07,401995,1,3.675641e+06,36585,0,0.064239,0.091009,3.297407e+06,31814,NTB_1,1,3.877800e+06,34250,0.850329,0.928876,103.646405,70927548.79,3.874375e+06,37380.702190
1,2018,D,5.721810e+07,401995,1,6.414162e+06,60238,2,0.112100,0.149848,1.074583e+06,4675,React__Omni,_Omni,6.781020e+06,56169,0.158469,0.083231,229.857276,70927548.79,1.259988e+06,5481.609191
2,2018,D,5.721810e+07,401995,1,6.414162e+06,60238,2,0.112100,0.149848,5.706438e+06,51494,React_1,1,6.781020e+06,56169,0.841531,0.916769,110.817524,70927548.79,6.691007e+06,60378.599053
3,2018,D,5.721810e+07,401995,1,4.712819e+07,305170,1,0.823659,0.759139,1.807705e+07,52128,Existing__Omni,_Omni,5.305491e+07,262268,0.340723,0.198759,346.781892,70927548.79,1.990510e+07,57399.469748
4,2018,D,5.721810e+07,401995,1,4.712819e+07,305170,1,0.823659,0.759139,3.497786e+07,210140,Existing_1,1,5.305491e+07,262268,0.659277,0.801242,166.450267,70927548.79,3.851501e+07,231390.485127


In [81]:
df1[['FISCAL_QTR', 'chan_rec__seg_custs']].groupby(['FISCAL_QTR']).agg('sum')

,chan_rec__seg_custs
FISCAL_QTR,
1,831070
2,895229
3,806497
4,903114


In [81]:
df1.rec_seg_exp_sales.sum()

680133057.3217437

In [92]:
df2 = pd.merge(df1, to_join, on = 'd_p_cust', how = 'inner')

In [96]:
df2['implied_custs'] = df2.rec_seg_exp_sales / df2.back_d_p_cust
df2.implied_custs.sum()

1942527.229670575

In [77]:
df2.tail()

,FISCAL_YR,Channel,total_net_sales,total_custs,FISCAL_QTR,recency_net_sales_total,recency_cust_total,Recency,chan_rec_sales_pct,chan_rec_cust_pct,chan_rec_seg_sales,chan_rec__seg_custs,Customer_Type,Segment,chan_rec_sales,chan_rec__custs,sales_pct,cust_pct,d_p_cust,channel_qtr_sales,rec_seg_exp_sales,implied_custs,back_d_p_cust,back_implied_custs
43,2018,R,1.151608e+08,537651,1,1.548893e+07,94888,2,0.134498,0.176486,1.378097e+07,86144,React_1,1,1.512208e+07,90213,0.911315,0.954896,159.975931,1.024696e+08,1.255972e+07,78510.076005,152.453708,82383.844044
44,2018,R,1.151608e+08,537651,1,8.767955e+06,55640,0,0.076137,0.103487,7.769443e+06,50869,NTB_1,1,8.565796e+06,53204,0.907031,0.956112,152.734334,1.024696e+08,7.076373e+06,46331.255630,145.458976,48648.585655
45,2018,R,1.151608e+08,537651,1,9.089780e+07,387094,1,0.789312,0.719973,1.845686e+07,42902,Existing__Omni,_Omni,8.497109e+07,334966,0.217213,0.128079,430.209805,1.024696e+08,1.756832e+07,40836.632780,428.189163,41029.342463
46,2018,R,1.151608e+08,537651,1,9.089780e+07,387094,1,0.789312,0.719973,6.651423e+07,292064,Existing_1,1,8.497109e+07,334966,0.782786,0.871921,227.738528,1.024696e+08,6.331213e+07,278003.608327,226.668878,279315.506904
47,2018,R,1.151608e+08,537651,1,8.767955e+06,55640,0,0.076137,0.103487,7.963530e+05,2335,NTB__Omni,_Omni,8.565796e+06,53204,0.092969,0.043888,341.050514,1.024696e+08,7.253146e+05,2126.707383,324.804879,2233.078054


In [74]:
yearly_data.shape

(12, 22)

In [14]:
year_end_fcst = yearly_data[['Channel', 'Recency', 'Customer_Type', 'Segment', 'implied_custs']]
year_end_fcst

,Channel,Recency,Customer_Type,Segment,implied_custs
0,D,0,NTB_1,1,148997.663044
1,D,0,NTB__Omni,_Omni,20520.120222
4,D,1,Existing_1,1,266491.397755
5,D,1,Existing__Omni,_Omni,161649.139806
8,D,2,React_1,1,221052.462513
9,D,2,React__Omni,_Omni,40838.961381
2,R,0,NTB_1,1,187412.719399
3,R,0,NTB__Omni,_Omni,18863.811405
6,R,1,Existing_1,1,367541.383056
7,R,1,Existing__Omni,_Omni,160016.121752


In [15]:
omni = year_end_fcst.query('Segment == "_Omni"')
omni

,Channel,Recency,Customer_Type,Segment,implied_custs
1,D,0,NTB__Omni,_Omni,20520.120222
5,D,1,Existing__Omni,_Omni,161649.139806
9,D,2,React__Omni,_Omni,40838.961381
3,R,0,NTB__Omni,_Omni,18863.811405
7,R,1,Existing__Omni,_Omni,160016.121752
11,R,2,React__Omni,_Omni,36733.887016


In [16]:
noms = pd.merge(omni, omni, on = 'Recency', how='inner')
noms = noms

In [19]:
noms

,Channel_x,Recency,Customer_Type_x,Segment_x,implied_custs_x,Channel_y,Customer_Type_y,Segment_y,implied_custs_y
0,D,0,NTB__Omni,_Omni,20520.120222,D,NTB__Omni,_Omni,20520.120222
1,D,0,NTB__Omni,_Omni,20520.120222,R,NTB__Omni,_Omni,18863.811405
2,R,0,NTB__Omni,_Omni,18863.811405,D,NTB__Omni,_Omni,20520.120222
3,R,0,NTB__Omni,_Omni,18863.811405,R,NTB__Omni,_Omni,18863.811405
4,D,1,Existing__Omni,_Omni,161649.139806,D,Existing__Omni,_Omni,161649.139806
5,D,1,Existing__Omni,_Omni,161649.139806,R,Existing__Omni,_Omni,160016.121752
6,R,1,Existing__Omni,_Omni,160016.121752,D,Existing__Omni,_Omni,161649.139806
7,R,1,Existing__Omni,_Omni,160016.121752,R,Existing__Omni,_Omni,160016.121752
8,D,2,React__Omni,_Omni,40838.961381,D,React__Omni,_Omni,40838.961381
9,D,2,React__Omni,_Omni,40838.961381,R,React__Omni,_Omni,36733.887016


In [37]:
oms = noms.query('Channel_x != Channel_y').copy()
oms['omni'] = oms.implied_custs_x + oms.implied_custs_y
oms = oms[['Customer_Type_x', 'omni']].copy()
oms = oms.drop_duplicates()

In [38]:
oms.omni.sum()

438622.0415827384

In [40]:
oms.head()

,Customer_Type_x,omni
1,NTB__Omni,39383.931627
5,Existing__Omni,321665.261558
9,React__Omni,77572.848397


In [47]:
melt.rename(columns={'Unnamed: 0':'MASTER_CHANNEL',
                      'Report in thousands.1': 'sales_segment',
                          'Unnamed: 4':'summation_type'
                          }, 
                 inplace=True)

We will do a quick aggregation at this point to confirm that the summations we've applied are working properly. We want to verify that these totals line up with the totals taken from Wes's workbook. The grand total should match the current sales forecast.

In [48]:
sub = melt[melt['summation_type'] == "Actual (TY) / Forecast"]
sub = sub[sub['sales_segment'].isin(['Direct Net Sales', 'Retail Sales (Stores & Concierge)'])]
sub.head()
sub[['MASTER_CHANNEL', 'SALES']].groupby('MASTER_CHANNEL').agg('sum')

,SALES
MASTER_CHANNEL,
Direct,2.782062e+08
RETAIL,4.019490e+08


In [35]:
sub.SALES.sum()

680155158.749575

In [36]:
sub.head()

,MASTER_CHANNEL,sales_segment,summation_type,FISCAL_WK,SALES
54,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,Feb Wk. 1 2019,5476461.31
135,Direct,Direct Net Sales,Actual (TY) / Forecast,Feb Wk. 1 2019,4089837.30
243,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,Feb Wk. 2 2019,4662842.01
324,Direct,Direct Net Sales,Actual (TY) / Forecast,Feb Wk. 2 2019,3747693.15
432,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,Feb Wk. 3 2019,5506713.02


Now the data is formatted appropriately so we are ready to move with the next step. Ultimately we need our forecasts at the monthly level, so our next task is to clean up the `FISCAL_WK` column so that we can use to to create a `FISCAL_MO` column.

In [49]:
sub.FISCAL_WK = sub.FISCAL_WK.str[8:]
sub.FISCAL_WK = sub.FISCAL_WK.str[:2]
sub.FISCAL_WK = sub.FISCAL_WK.str.strip()
sub.head()

,MASTER_CHANNEL,sales_segment,summation_type,FISCAL_WK,SALES
54,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,1,5476461.31
135,Direct,Direct Net Sales,Actual (TY) / Forecast,1,4089837.30
243,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,2,4662842.01
324,Direct,Direct Net Sales,Actual (TY) / Forecast,2,3747693.15
432,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,3,5506713.02


In [50]:
sub.FISCAL_WK.unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45',
       '46', '47', '48', '49', '50', '51', '52'], dtype=object)

In [51]:
h1 = [1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6]
h2 = [x+1 for x in h1]

months = h1 + h2
weeks = list(range(1, 53))

month_map = pd.DataFrame()
month_map['FISCAL_MO'] = months
month_map['FISCAL_WK'] = weeks

month_map.head()

,FISCAL_MO,FISCAL_WK
0,1,1
1,1,2
2,1,3
3,1,4
4,2,5


In [52]:
month_map.shape

(52, 2)

In [53]:
sub.dtypes

MASTER_CHANNEL     object
sales_segment      object
summation_type     object
FISCAL_WK          object
SALES             float64
dtype: object

In [54]:
sub.FISCAL_WK = sub.FISCAL_WK.astype(str).astype(int)
sales_fcst = pd.merge(sub, month_map, on='FISCAL_WK', how='inner')
sales_fcst.head()

,MASTER_CHANNEL,sales_segment,summation_type,FISCAL_WK,SALES,FISCAL_MO
0,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,1,5476461.31,1
1,Direct,Direct Net Sales,Actual (TY) / Forecast,1,4089837.30,1
2,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,2,4662842.01,1
3,Direct,Direct Net Sales,Actual (TY) / Forecast,2,3747693.15,1
4,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,3,5506713.02,1


In [55]:
sales_fcst.head(26)

,MASTER_CHANNEL,sales_segment,summation_type,FISCAL_WK,SALES,FISCAL_MO
0,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,1,5476461.31,1
1,Direct,Direct Net Sales,Actual (TY) / Forecast,1,4089837.30,1
2,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,2,4662842.01,1
3,Direct,Direct Net Sales,Actual (TY) / Forecast,2,3747693.15,1
4,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,3,5506713.02,1
5,Direct,Direct Net Sales,Actual (TY) / Forecast,3,2806510.55,1
6,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,4,7207303.66,1
7,Direct,Direct Net Sales,Actual (TY) / Forecast,4,6487624.60,1
8,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,5,6706428.53,2
9,Direct,Direct Net Sales,Actual (TY) / Forecast,5,4296148.16,2


We might want to work with the LY , plan, or other summation types, but for this notebook we are only interested in the `Actual (TY) / Forecast` sales totals. So now we are ready to subset this data to just forecasted sales so we can get to work on building together a customer forecast.

In [58]:
fcst = sales_fcst.query('summation_type == "Actual (TY) / Forecast"')
fcst.head()

,MASTER_CHANNEL,sales_segment,summation_type,FISCAL_WK,SALES,FISCAL_MO
0,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,1,5476461.31,1
1,Direct,Direct Net Sales,Actual (TY) / Forecast,1,4089837.30,1
2,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,2,4662842.01,1
3,Direct,Direct Net Sales,Actual (TY) / Forecast,2,3747693.15,1
4,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,3,5506713.02,1


In [57]:
fcst.shape

(104, 3)